In [1]:
# Optical Ray Tracing & Spot Distribution Demo for Materials/Energy Research
# 光学追迹与光斑分布交互式模拟演示（面向材料/电池应用）
# ----------------------------------------------
# Author: lunazhang

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from ipywidgets import interact, IntSlider, FloatSlider, Layout
import warnings

warnings.filterwarnings('ignore')

"""
该 Notebook 用于演示多镜组下光线路径与光斑分布对材料界面的影响。
This notebook demonstrates interactive ray tracing and spot pattern analysis for multi-mirror optical systems, targeting materials and battery interface research.
"""


'\n该 Notebook 用于演示多镜组下光线路径与光斑分布对材料界面的影响。\nThis notebook demonstrates interactive ray tracing and spot pattern analysis for multi-mirror optical systems, targeting materials and battery interface research.\n'

In [2]:
# =================== 材料参数与镜组参数、光线追迹主函数 ===================

# ------ 材料参数接口（可扩展）/ Material property interface (extendable) ------
MATERIAL_DEFAULTS = {
    "refractive_index": 1.5,   # 折射率 Refractive index (n)
    "absorption": 0.05,        # 吸收率 Absorption coefficient (arbitrary unit)
    # 可拓展更多属性 e.g. 电导率、厚度等
}

# ------ 光源参数 / Light source params ------
SOURCE_POS = np.array([0, 0])         # 光源坐标 (x0, y0)
N_RAYS = 200                          # 光线数量 Number of rays
THETA_SPREAD = np.deg2rad(10)         # 初始发散角 spread (radians)
SOURCE_RADIUS = 0.2                   # 光源近似直径

# ------ 镜子/透镜组结构 Mirror/Lens group definition ------
def default_mirrors(num_mirrors):
    """
    生成默认镜组参数：等间隔摆放、均匀角度
    Generate default mirror params: evenly spaced, equal angle
    """
    mirror_list = []
    RADIUS = 3.0
    for i in range(num_mirrors):
        angle = np.pi / 6 + (i / max(1,num_mirrors-1)) * np.pi / 2  # 30°起，等间距
        pos = np.array([RADIUS * np.cos(angle), RADIUS * np.sin(angle)])
        normal_angle = angle - np.pi / 2  # 法线方向
        mirror_list.append({"pos": pos, "angle": normal_angle})
    return mirror_list

# =================== 光线追迹主函数 / Ray tracing ===================

def reflect_ray(ray_origin, ray_dir, mirror_pos, mirror_angle):
    """
    计算光线与镜子反射后的新方向
    Compute reflected ray direction after hitting mirror
    """
    n = np.array([np.cos(mirror_angle), np.sin(mirror_angle)])
    incident = ray_dir / np.linalg.norm(ray_dir)
    reflect = incident - 2 * np.dot(incident, n) * n
    return reflect

def trace_rays(mirrors, source_pos, n_rays, theta_spread, max_bounces=1):
    """
    跟踪每条光线在多镜组下的传播与反射轨迹
    Trace rays through multi-mirror system (single bounce)
    """
    rays = []
    theta_0 = np.pi/2   # 默认垂直发射
    for i in range(n_rays):
        theta = theta_0 + np.random.uniform(-theta_spread/2, theta_spread/2)
        ray_pos = source_pos.copy()
        ray_dir = np.array([np.cos(theta), np.sin(theta)])
        points = [ray_pos.copy()]
        for mirror in mirrors:
            m_pos, m_angle = mirror["pos"], mirror["angle"]
            to_mirror = m_pos - ray_pos
            distance = np.dot(to_mirror, ray_dir)
            impact = ray_pos + ray_dir * distance
            points.append(impact)
            ray_dir = reflect_ray(ray_pos, ray_dir, m_pos, m_angle)
            ray_pos = impact
        SCREEN_Y = 5.0
        if ray_dir[1] != 0:
            t = (SCREEN_Y - ray_pos[1]) / ray_dir[1]
            end = ray_pos + ray_dir * t
        else:
            end = ray_pos + ray_dir * 10
        points.append(end)
        rays.append(points)
    return rays


In [3]:
# =================== 光斑分布与发散角计算 / Spot distribution & divergence ===================

def get_spot_on_screen(rays, screen_y=5.0):
    """
    统计所有光线在屏幕上的命中点
    Collect hit points of all rays at the sample/screen (y=screen_y)
    """
    hits = []
    for ray in rays:
        pt1, pt2 = ray[-2], ray[-1]
        if pt2[1] == pt1[1]:
            continue
        t = (screen_y - pt1[1]) / (pt2[1] - pt1[1])
        x_hit = pt1[0] + t * (pt2[0] - pt1[0])
        hits.append([x_hit, screen_y])
    return np.array(hits)

def plot_spot_heatmap(hits, bins=60, show=True):
    """
    绘制光斑分布热力图
    """
    if hits.shape[0] < 2:
        print("无足够命中点！Not enough hit points!")
        return
    plt.figure(figsize=(5,3.5))
    plt.hist(hits[:,0], bins=bins, alpha=0.7, color='royalblue', label="X spots")
    plt.xlabel("Screen X Position / 屏幕X坐标")
    plt.ylabel("Intensity / 光斑强度")
    plt.title("Spot Intensity Distribution on Screen / 屏幕光斑强度分布")
    plt.tight_layout()
    if show:
        plt.show()

def calc_divergence(hits):
    """
    计算光束发散角（近似为光斑宽度/距离）
    Compute beam divergence (FWHM of spot, approx.)
    """
    if hits.shape[0]<2:
        return 0
    x_sorted = np.sort(hits[:,0])
    low, high = np.percentile(x_sorted, [5,95])
    divergence = np.arctan((high-low)/5.0) # 5.0为屏幕y坐标
    return np.degrees(divergence)


In [4]:
# =================== 主绘图与交互控件 / Main visualization & interactivity ===================

def draw_system_and_spots(num_mirrors=2, theta_spread_deg=10, refractive_index=1.5, absorption=0.05):
    """
    绘制整个系统：镜组、光线路径、屏幕光斑分布
    Draw whole system: mirrors, rays, spot distribution
    """
    mirrors = default_mirrors(num_mirrors)
    rays = trace_rays(mirrors, SOURCE_POS, N_RAYS, np.deg2rad(theta_spread_deg))
    hits = get_spot_on_screen(rays)

    plt.figure(figsize=(7,5))
    for m in mirrors:
        pos, angle = m["pos"], m["angle"]
        dx, dy = 0.3*np.cos(angle+np.pi/2), 0.3*np.sin(angle+np.pi/2)
        plt.plot([pos[0]-dx, pos[0]+dx], [pos[1]-dy, pos[1]+dy], 'k-', lw=4)
    for pts in rays:
        pts = np.array(pts)
        plt.plot(pts[:,0], pts[:,1], color='orange', alpha=0.15)
    plt.gca().add_patch(Circle(SOURCE_POS, SOURCE_RADIUS, color='yellow', alpha=0.4, label="Source"))
    plt.axhline(y=5.0, color='blue', linestyle='--', label="Screen")
    plt.xlim(-4, 4)
    plt.ylim(-1, 6)
    plt.xlabel("X / mm")
    plt.ylabel("Y / mm")
    plt.title("Multi-mirror Optical Ray Tracing System 多镜组光学追迹系统")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plot_spot_heatmap(hits, show=True)
    div_angle = calc_divergence(hits)
    print(f"光束发散角 (近似): {div_angle:.2f}°  (Beam divergence, approx.)")
    print(f"光斑宽度（屏幕上90%覆盖范围）：{np.ptp(hits[:,0]):.2f} mm")

print("👇请选择镜子数量、光源发散角度、材料参数等，自动展示光斑分布和发散角。")
interact(draw_system_and_spots,
         num_mirrors=IntSlider(min=1, max=6, step=1, value=2, description="镜子数量/Num Mirrors"),
         theta_spread_deg=FloatSlider(min=2, max=30, step=1, value=10, description="光源发散角 (°)"),
         refractive_index=FloatSlider(min=1.0, max=2.5, step=0.05, value=1.5, description="折射率n"),
         absorption=FloatSlider(min=0, max=0.3, step=0.01, value=0.05, description="吸收率Abs."),
        )


👇请选择镜子数量、光源发散角度、材料参数等，自动展示光斑分布和发散角。


interactive(children=(IntSlider(value=2, description='镜子数量/Num Mirrors', max=6, min=1), FloatSlider(value=10.0…

<function __main__.draw_system_and_spots(num_mirrors=2, theta_spread_deg=10, refractive_index=1.5, absorption=0.05)>

In [6]:
# =================== 预留材料/界面参数接口说明 ===================
"""
【接口说明 Interface Doc】
- 镜组参数、材料参数均为可扩展输入：可改写default_mirrors等函数，实现自定义镜组结构/材料分布。
- 预留折射率、吸收率等参数，进一步对接多层膜/异质结/纳米结构的物理模型。
- 可通过上传CSV/Excel或手动输入实现材料参数批量加载。
- 可拓展与COMSOL/FDTD等仿真结果联动。
"""


'\n【接口说明 Interface Doc】\n- 镜组参数、材料参数均为可扩展输入：可改写default_mirrors等函数，实现自定义镜组结构/材料分布。\n- 预留折射率、吸收率等参数，进一步对接多层膜/异质结/纳米结构的物理模型。\n- 可通过上传CSV/Excel或手动输入实现材料参数批量加载。\n- 可拓展与COMSOL/FDTD等仿真结果联动。\n'